# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Изучение общей информации

In [2]:
import pandas as pd
data = pd.read_csv('C:/Users/79833/Desktop/Data/1 project/data.csv')
pd.set_option('display.float_format', '{:.2f}'.format)
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.67,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,покупка жилья
1,1,-4024.80,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,приобретение автомобиля
2,0,-5623.42,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,покупка жилья
3,3,-4124.75,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,дополнительное образование
4,0,340266.07,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,сыграть свадьбу


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Итого файл состоит из 12 столбцов и 21525 строк. Присутствуют типы данных: *float64* - 2 столбца, *int64*  - 5 столбцов, *object*- 5 столбцов.

Посмотрю названия столбцов:

In [3]:
print(data.columns)

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')


Подробно разберу каждый:
   - children — количество детей в семье;
   - days_employed — общий трудовой стаж в днях;
   - dob_years — возраст клиента в годах;
   - education — уровень образования клиента;
   - education_id — идентификатор уровня образования;
   - family_status — семейное положение;
   - family_status_id — идентификатор семейного положения;
   - gender — пол клиента;
   - income_type — тип занятости;
   - debt — имел ли задолженность по возврату кредитов;
   - total_income — ежемесячный доход;
   - purpose — цель получения кредита.

## Предобработка данных

### Обработка пропусков

Для начала рассчитываю количество пропущенных значений методом *isnull()* и посчитаю их количество с помощью *sum()*:

In [4]:
print(data.isnull().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


В столбцах 'days_employed' - 'общий трудовой стаж в днях' и 'total_income' - 'ежемесячный доход' по 2174 пропущенных значения. Можно предположить, что данные с пропусками в вышеуказанных столбцах являются данными о людях с зарплатными проектами в банке. Другими словами, можно предположить, что если если потенциальный заёмщик получает зарплату/пенсию на счет в банке, для одобрения кредита ему уже не нужно подавать дополнительные документы, например, 2-НДФЛ, так как банк уже располагает этими данными. Однако возникает вопрос почему эти данные не "подтянулись" в анализируемую таблицу.

Для замены пустых значений общего трудового стажа ('days_employed') использую медианные значения.

Посмотрю уникальные значения колонки 'income_type' с помощью unique() и занесу их в отдельный лист income_types:

In [5]:
income_types = data['income_type'].unique()
print(income_types)

['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


Отсортирую исходные данные по возрастанию значений в колонке 'total_income' с помощью sort_values():

In [6]:
data = data.sort_values(by='total_income')

Для поиска медианы по всем типам занятости создам цикл с помощью оператора *for*: 

In [7]:
for type in income_types:
    print(type, data.loc[data['income_type']==type, 'total_income'].median())

сотрудник 142594.39684740017
пенсионер 118514.48641164352
компаньон 172357.95096577113
госслужащий 150447.9352830068
безработный 131339.7516762103
предприниматель 499163.1449470857
студент 98201.62531401133
в декрете 53829.13072905995


Для дальнейшего исследования создам словарь *income_medians* и с помощью цикла добавлю в него данные о типах занятости

In [8]:
income_medians = {}
for type in income_types:
    income_medians[type] = data.loc[data['income_type']==type, 'total_income'].median()
print(income_medians)

{'сотрудник': 142594.39684740017, 'пенсионер': 118514.48641164352, 'компаньон': 172357.95096577113, 'госслужащий': 150447.9352830068, 'безработный': 131339.7516762103, 'предприниматель': 499163.1449470857, 'студент': 98201.62531401133, 'в декрете': 53829.13072905995}


Обработаю пропуски в колонке *'total_income'*, заменив в ней все пустые значения на искусственное значение -1 с помощью *fillna()*: 

In [9]:
data['total_income'] = data['total_income'].fillna(-1)

Создам функцию *default_income*, в которой заменю значение -1 на медианное значение в соответствии с типом занятости из словаря *income_medians*. Далее применю эту функцию для всех значений колонки *'total_income'* с помощью  метода *apply()* 

In [10]:
def default_income(row):
    if row['total_income'] == -1:
        row['total_income'] = income_medians[row['income_type']]
    return row
  
data['total_income'] = data.loc[:, ['total_income', 'income_type']].apply(default_income, axis=1)['total_income']

После проделланых операций в колонке *'total_income'* не осталось пропусков, все пропущенные значения заменены на средние соответсвенного источника дохода.

Выведу первые 5 значений из колонки *'days_employed'*:

In [11]:
print(data['days_employed'].head())

14585    359219.059341
13006    369708.589113
16174     -3642.820023
1598     359726.104207
14276    346602.453782
Name: days_employed, dtype: float64


Очевидно, что данные в колонке *'days_employed'* указаны некорректно, так как трудовой стаж в днях не может принимать настолько большие и отрицательные значения. Так как для дальнейшего исследования данные о стаже не используются, просто заменю пропуски на 0 с помощью *fillna()*:

In [12]:
data['days_employed'] = data['days_employed'].fillna(0)

Проверю наличие пропусков методом *isnull()* и посчитаю их количество с помощью *sum()*

In [13]:
print(data.isnull().sum())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


### Замена типа данных

Проверю типы данных всех столбов:

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 14585 to 21510
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


Для замены вещественного типа данных на целочисленный применю цикл, меняющий тип данных *float* на *int* с помощью метода *astype()*:

In [15]:
for i in data.columns:
    if data[i].dtype == float:
        data[i] = data[i].astype('int')

Проверю результат замены:

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 14585 to 21510
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


**Вывод**

### Обработка дубликатов

До начала поиска дубликатов приведу все символы к нижнему регистру вызовом метода *str.lower()* в теле цикла:

In [17]:
for i in data.columns:
    if data[i].dtype == object:
        data[i] = data[i].str.lower()

Посчитаю количество дубликатов с помошью метода duplicated():

In [18]:
print(data.duplicated().sum())

71


всего найдено 71 дубликат. Причиной появления дубликатов может стать как ошибочное повторное добавление тех же самых данных, так и разница в раскладке регистра при заполнении данных

Для удаления дубликатов использую метод *drop_duplicates()*:

In [19]:
data = data.drop_duplicates().reset_index(drop=True)

Проверяю на отсутствие дубликатов:

In [20]:
print(data.duplicated().sum())

0


**Вывод**

### Лемматизация

Для начала посмотрю список целей на получение кредита и их количество с помощью *value_counts()* и отсортирую по алфавиту названия целей с помощью *sort_index()*:

In [21]:
print(data['purpose'].value_counts().sort_index())

автомобили                                478
автомобиль                                494
высшее образование                        452
дополнительное образование                460
жилье                                     646
заняться высшим образованием              496
заняться образованием                     408
на покупку автомобиля                     471
на покупку подержанного автомобиля        478
на покупку своего автомобиля              505
на проведение свадьбы                     768
недвижимость                              633
образование                               447
операции с жильем                         652
операции с коммерческой недвижимостью     650
операции с недвижимостью                  675
операции со своей недвижимостью           627
покупка жилой недвижимости                606
покупка жилья                             646
покупка жилья для сдачи                   651
покупка жилья для семьи                   638
покупка коммерческой недвижимости 

Из представленных данных видно, что во многих строках цель кредита одна и та же, но записана разными формулировками, например, 'свадьба', 'на проведение свадьбы', 'сыграть свадьбу'.  

Посчитаю вцелом количество уникальных значений в столбце 'purpose':

In [22]:
print(len(data['purpose'].unique()))

38


Всего 38 причин для взятия кредита.

Для того, чтобы сократить количество целей для получения кредита использую лемматизацию, которая поможет привести слова в инфинитив.
Импортирую библиотеку pymystem3.
Создам новый столбец *'lemmas'*, в который вернуться цели кредита после лемматизации. Для этого применю метод *lemmatize* ко всем значениям колонки *'purpose'* с помощью метода *apply()*.

Выведу первые 5 значений новой колонки *'lemmas'*.

In [23]:
from pymystem3 import Mystem
m=Mystem()

data['lemmas'] = data['purpose'].apply(m.lemmatize)

print(data['lemmas'].head())

0                              [недвижимость, \n]
1    [заниматься,  , высокий,  , образование, \n]
2               [приобретение,  , автомобиль, \n]
3             [на,  , проведение,  , свадьба, \n]
4                              [недвижимость, \n]
Name: lemmas, dtype: object


Для подсчета количества используемых слов после лемматизации использую *Counter* библиотеки *collections*:

In [24]:
from collections import Counter
print(Counter(data['lemmas'].sum()))

Counter({' ': 33570, '\n': 21454, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


Так как многие слова в инфинитиве употребляются в разных словосочетаниях, объединю их в 4 группы с помощью объявления функции *map_lemma* и применения ее ко всем значениям колонки *'lemmas'* методом *apply()*. Выведу первые 10 строк обновленной колонки:

In [25]:
def map_lemma(row):
    if 'свадьба' in row:
        return 'свадьба'
    if 'недвижимость' in row or 'жилье' in row:
        return 'недвижимость'
    if 'автомобиль' in row:
        return 'автомобиль'
    if 'образование' in row:
        return 'образование'
    return row
    
data['lemmas'] = data['lemmas'].apply(map_lemma)
print(data['lemmas'].head(10))  


0    недвижимость
1     образование
2      автомобиль
3         свадьба
4    недвижимость
5    недвижимость
6    недвижимость
7    недвижимость
8     образование
9     образование
Name: lemmas, dtype: object


Проверю уникальные значения в колонке 'lemmas':

In [26]:
print(data['lemmas'].unique())

['недвижимость' 'образование' 'автомобиль' 'свадьба']


**Вывод**

Лемматизация помогает категоризировать текстовые данные для эффективного анализа

### Категоризация данных

Проверю уникальные значения и их количество в колонке 'children' с помощью value_counts()

In [27]:
print(data['children'].value_counts())

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


 Очевидно, что -1 ребенка быть не может и 20 тоже скорее всего ошибка

Заменю значение -1 на 1 и 20 на 2, предполагая опечатку:

In [28]:
def child(row):
    if row == -1:
        return 1
    if row == 20:
        return 2
    return row


data['children'] = data['children'].apply(child)

Повторно посмотрю уникальные значения в колонке 'children':

In [29]:
print(data['children'].unique())

[0 1 2 3 5 4]


Для категоризации дохода заемщиков посмотрю размер минимального и максимального дохода с помощью min() и max():

In [30]:
print('Минимальное значение ежемесячного дохода:', data['total_income'].min())
print('Максимальное значение ежемесячного дохода:', data['total_income'].max())

Минимальное значение ежемесячного дохода: 20667
Максимальное значение ежемесячного дохода: 2265604


Далее создам цикл, разбивающий заемщиков по нескольким категорям дохода:

In [31]:
def income(row):
    if row < 100000:
        return 'меньше 100 тыс.'
    if 100000 <= row < 200000:
        return 'от 100 до 200 тыс.'
    if 200000 <= row < 300000:
        return 'от 200 до 300 тыс.'
    if 300000 <= row < 400000:
        return 'от 300 до 400 тыс.'
    if 400000 <= row < 500000:
        return 'от 400 до 500 тыс.'
    if row >= 500000:
        return 'от 500 тыс.'
    return 'доход не указан'

data['income_level'] = data['total_income'].apply(income)

Проверю данные о доходе после категоризации:

In [32]:
print(data['income_level'].unique())

['меньше 100 тыс.' 'от 100 до 200 тыс.' 'от 200 до 300 тыс.'
 'от 300 до 400 тыс.' 'от 400 до 500 тыс.' 'от 500 тыс.']


**Вывод**

Категоризация данных позволила исключить ошибки, возникшие в результате человеческого фактора в колонке с количеством детей заемщиков, а также разбить на группы заемщиков по количеству детей и по уровню дохода для дальнейщего более эффективного анализа.

## Исследовательский анализ

Для дальнейшего анализа рассчитаю конверсию по всей колонке 'debt'

In [33]:
avg = data['debt'].mean()
print('Общая средняя конверсия задолженностей по кредитам:', avg)

Общая средняя конверсия задолженностей по кредитам: 0.08115036822970076


В последующем анализе зависимость между возвратом кредита в срок и различными категорями заемщиков буду рассчитывать по одной схеме: 

* с помощью *groupby()* и *agg()* создам сводную таблицу с информацией о количестве кредитов по каждой категории (подколонка *'count'*) и количестве невозвратов в срок (подколонка *'sum'*)

* посчитаю по каждой категории конверсию задолженностей и отсортирую конверсию по убыванию (т.е. от наибольшей доли невозвратов к меньшей)

* в колонке *'delta'* будет указана разница между конверсией по категории и общей средней конверсии по задолженности
    

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [34]:
children = data.groupby('children').agg({'debt': ['count', 'sum']})

children['conversion'] = children['debt']['sum']/children['debt']['count']

children['delta'] = children['conversion'] - avg

children = children.sort_values('conversion', ascending=False)

print(children)


           debt       conversion     delta
          count   sum                     
children                                  
4            41     4   0.097561  0.016411
2          2128   202   0.094925  0.013774
1          4855   445   0.091658  0.010508
3           330    27   0.081818  0.000668
0         14091  1063   0.075438 -0.005712
5             9     0   0.000000 -0.081150


**Вывод**

Для анализа семьи с 4 и 5 детьми выборка составляет всего 41 и 9 заемщиков соответсвенно, что не позволяет сделать однозначного вывода? Но можно отметить, что наибольшую долю невозвратов кредитов в срок имеют семьи с 4 детьми, среди них возвращают кредиты не в срок почти 10% заемщиков, а семьи с 5 детьми не имеют просрочки по кредитам.
Заемщики с 1-2 детьми состовляют в сумме выборку из 6983 чел. могут иметь задолженности по кредитам чаще среднего на 1-1,3%.
Самая многочисленная категория - заемщики без детей (14091 чел.). Среди них возвращают кредиты не в срок 7,5%, что на половину процента меньше чем общая средняя конверсия невозвратов.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [35]:
marital_status = data.groupby('family_status').agg({'debt': ['count', 'sum']})

marital_status['conversion'] = marital_status['debt']['sum']/marital_status['debt']['count']

marital_status['delta'] = marital_status['conversion'] - avg

marital_status = marital_status.sort_values('conversion', ascending=False)

print(marital_status)

                        debt      conversion     delta
                       count  sum                     
family_status                                         
не женат / не замужем   2810  274   0.097509  0.016359
гражданский брак        4151  388   0.093471  0.012321
женат / замужем        12339  931   0.075452 -0.005699
в разводе               1195   85   0.071130 -0.010021
вдовец / вдова           959   63   0.065693 -0.015457


**Вывод**

Наибольшее число кредитов приходится на заемщиков, состоящих в браке (12339 чел.). Среди них 7,54% не возвращают в срок кредиты, что на 1.6% реже среднего значения.
Из всех категорий имеют конверсию невозврата выше средней только заемщики, не состоящие в браке или живущие в гражданском: 9,75% и 9,34% соответственно.
Самая малочисленная группа 'вдовец/вдова' имеют наименьшее количество невозвратов - 6,56%.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [36]:
income = data.groupby('income_level').agg({'debt': ['count', 'sum']})

income['conversion'] = income['debt']['sum']/income['debt']['count']

income['delta'] = income['conversion'] - avg

income = income.sort_values('conversion', ascending=False)

print(income)

                     debt       conversion     delta
                    count   sum                     
income_level                                        
от 100 до 200 тыс.  11924  1029   0.086297  0.005146
меньше 100 тыс.      4463   354   0.079319 -0.001832
от 300 до 400 тыс.    954    75   0.078616 -0.002534
от 200 до 300 тыс.   3584   252   0.070312 -0.010838
от 500 тыс.           222    14   0.063063 -0.018087
от 400 до 500 тыс.    307    17   0.055375 -0.025776


**Вывод**

Наименьшее количество возвратов кредитов не срок имеют заемщики, относящиеся к категорям дохода 'от 500 тыс.' и 'от 400 до 500 тыс.': 6,3% и 5,5% соответственно.
Наибольшее количество кредитов и наибольшее количество невозвратов в срок имеют заемщики, получающие в месяц от 100 до 200 тыс.руб. - их 11924 чел., на которых приходится 8,6% просрочек.

- Как разные цели кредита влияют на его возврат в срок?

In [37]:
purpose = data.groupby('lemmas').agg({'debt': ['count', 'sum']})

purpose['conversion'] = purpose['debt']['sum']/purpose['debt']['count']

purpose['delta'] = purpose['conversion'] - avg

purpose = purpose.sort_values('conversion', ascending=False)

print(purpose)

               debt      conversion     delta
              count  sum                     
lemmas                                       
автомобиль     4306  403   0.093590  0.012440
образование    4013  370   0.092200  0.011050
свадьба        2324  186   0.080034 -0.001116
недвижимость  10811  782   0.072334 -0.008817


**Вывод**

Наибольшее количество невозвратов в срок приходится на заемщиков, взявших кредит на автомобиль - 9,4%. Немного меньше задолжников среди тех, кто брал заемные средства для оплаты образования (на 0,1%).
С целью приобретения или ремонта недвижимости кредит взяли 10811 чел., среди которых 782 чел. вернули его не в срок, что состовляет самую большую выборку по всему массиву данных, с конверсией невозврата ниже средней - 7,2%.

## Общий вывод

- В результате исследования было выявлено, что в среднем возвращают кредит не в срок 8,1% заемщиков.
- Наибольшее число кредитов приходится на заемщиков, не имеющих детей, среди которых 7,5% не вернули в срок заемные средства. 9 семей с пятью детьми вообще не имели просрочки по кредиту, однако выборка для таких заемщиков очень мала.
- Также 7,5% невозвратов имеет самая крупная категория заемщиков, состоящих в браке. Наибольшее количество возвратов кредитов не в срок среди данных категорий имеют заемщики, не состоящие в официальных отношениях - в среднем 9,6%.
- Также было выявлено, что заемщики, получающие в месяц от 100 до 200 тыс.руб. чаще берут кредит, но 8,6% из них возвращают кредит не в срок. Заемщики с уровнем ежемесячного дохода от 400 тыс.руб. имеют конверсию меньше 6,3%. Соответственно люди, получающие меньше 100 тыс.руб. наравне с теми, кото зарабатывает от 200 до 400 тыс.руб. имеют почти одинаковую конверсию от 7 до 8%.
- Цель для получения кредита 'недвижимость' составляет половину от всех указываемых целей. По ней же самый маленький процент невозврата в срок - 7,2. Также конверсию ниже среднего имеет категория заемщиков, взявшая кредит на свадьбу - 8%. Заемщики, приобетающие на кредитные деньги автомобиль или его ремонт не возвращают в срок в 9,4% случаев.
